In [23]:
from keras.models import Model
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.pooling import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [7]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1./255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/Images/training/',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224x224
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/Images/test/',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 35850 images belonging to 2 classes.
Found 15367 images belonging to 2 classes.


In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [9]:
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
x = base_model.output
x = Flatten()(x)
x = Dense(256)(x)
x = Dense(1)(x)
x = Activation('sigmoid')(x)

In [12]:
model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=35850 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=15367 // batch_size)
model.save_weights('vgg16_notdog.h5')

Epoch 1/5
2240/2240 [==============================] - 1674s - loss: 0.2229 - acc: 0.9084 - val_loss: 0.1858 - val_acc: 0.9258
Epoch 2/5
2240/2240 [==============================] - 1667s - loss: 0.1602 - acc: 0.9365 - val_loss: 0.1645 - val_acc: 0.9369
Epoch 4/5
2240/2240 [==============================] - 1668s - loss: 0.1520 - acc: 0.9408 - val_loss: 0.2060 - val_acc: 0.9118
Epoch 5/5
2240/2240 [==============================] - 1667s - loss: 0.1425 - acc: 0.9435 - val_loss: 0.1613 - val_acc: 0.9353


In [ ]:
model.summary()

In [2]:
from keras.applications.xception import Xception

In [34]:
batch_size = 10

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1./255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/Images/training/',  # this is the target directory
        target_size=(299, 299),  # all images will be resized to 224x224
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/Images/test/',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='binary')

Found 35850 images belonging to 2 classes.
Found 15367 images belonging to 2 classes.


In [35]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [36]:
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [37]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1000)(x)
x = Dense(1)(x)
x = Activation('sigmoid')(x)

In [38]:
model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_4[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [40]:
model.fit_generator(
        train_generator,
        steps_per_epoch=35850 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=15367 // batch_size)
model.save_weights('xception_notdog.h5')

Epoch 1/5
3585/3585 [==============================] - 2169s - loss: 0.3513 - acc: 0.8948 - val_loss: 0.2114 - val_acc: 0.9691
Epoch 2/5
3585/3585 [==============================] - 2168s - loss: 0.1531 - acc: 0.9752 - val_loss: 0.1248 - val_acc: 0.9811
Epoch 3/5
3585/3585 [==============================] - 2168s - loss: 0.1063 - acc: 0.9812 - val_loss: 0.0923 - val_acc: 0.9851
Epoch 4/5
3585/3585 [==============================] - 2168s - loss: 0.0830 - acc: 0.9833 - val_loss: 0.0751 - val_acc: 0.9874
Epoch 5/5
3585/3585 [==============================] - 2168s - loss: 0.0731 - acc: 0.9845 - val_loss: 0.0624 - val_acc: 0.9890


In [20]:
full_model = Xception(weights='imagenet', include_top=True, input_shape=(299, 299, 3))

90923008/91884032 [============================>.] - ETA: 0s

In [21]:
full_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________